# SPR 2026 - TF-IDF + LinearSVC

**LinearSVC: SVM otimizado para dados lineares**

Vantagens:
- Extremamente rápido para dados esparsos
- Bom para classificação de texto (usa liblinear)
- Tempo de execução: **< 1 minuto**

---
**CONFIGURAÇÃO OFFLINE:**
1. No Kaggle, Settings → Internet → **OFF**
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
import warnings
warnings.filterwarnings('ignore')

SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# TF-IDF
tfidf = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)

X_train = tfidf.fit_transform(train['report'])
X_test = tfidf.transform(test['report'])
y_train = train['target'].values

print(f'TF-IDF shape: {X_train.shape}')

In [ ]:
# LinearSVC
model = LinearSVC(
    C=1.0,
    class_weight='balanced',
    max_iter=2000,
    random_state=SEED,
    dual=True  # dual=True é mais rápido quando n_samples < n_features
)

print('Treinando LinearSVC...')
model.fit(X_train, y_train)
print('Modelo treinado!')

In [ ]:
# Predições e submissão
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())